In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyrender
from sklearn.neighbors import NearestNeighbors
from scipy import optimize

from face_reconstruction.data.biwi import BiwiDataLoader
from face_reconstruction.model import BaselFaceModel
from face_reconstruction.landmarks import load_bfm_landmarks, detect_landmarks
from face_reconstruction.graphics import draw_pixels_to_image, register_rgb_depth, backproject_points, interpolate_around, SimpleImageRenderer, setup_standard_scene, get_perspective_camera 
from face_reconstruction.optim import BFMOptimization, run_icp, NearestNeighborMode, DistanceType, nearest_neighbors
from face_reconstruction.utils.math import add_column

# 1. Face Model

In [ ]:
bfm = BaselFaceModel.from_h5("model2019_face12.h5")
bfm_landmarks = load_bfm_landmarks("model2019_face12_landmarks_v2")
bfm_landmark_indices = np.array(list(bfm_landmarks.values()))

In [ ]:
n_shape_coefficients = bfm.get_n_shape_coefficients()
n_expression_coefficients = bfm.get_n_expression_coefficients()
n_color_coefficients = bfm.get_n_color_coefficients()

# 2. Input RGB-D Image 

In [ ]:
run_id = 1
frame_id = 4

biwi_loader = BiwiDataLoader(run_id)
frame = biwi_loader.get_frame(frame_id)

In [ ]:
img = frame.get_color_image()
img_width = biwi_loader.get_image_width()
img_height = biwi_loader.get_image_height()

In [ ]:
# Registration
pointcloud, colors, screen_positions = register_rgb_depth(frame.get_depth_image(), frame.get_color_image(), biwi_loader.get_depth_intrinsics(), biwi_loader.get_rgb_intrinsics(), biwi_loader.get_rgb_extrinsics())
pointcloud[:, 2] = -pointcloud[:, 2]  # Invert z-coordinate for easier rendering (point cloud will be right in front of camera)

# 3. Detect 3D Landmarks

In [ ]:
landmarks_img, face_pos = detect_landmarks(img, return_face_pos=True)
face_pos = face_pos[0] # Assume there is only one face

In [ ]:
# Create a depth image for easier querying of depth values
rgb_depth_img = np.zeros((img_height, img_width))
for point, screen_position in zip(pointcloud, screen_positions):
    rgb_depth_img[screen_position[1], screen_position[0]] = -point[2]

In [ ]:
# As RGB and depth channels are not aligned, we might not have exact depth information for every pixel in the color channel. Hence, we have to interpolate
interpolation_size = 1
rgb_depth_values = [interpolate_around(rgb_depth_img, pixel, interpolation_size) for pixel in landmarks_img]

In [ ]:
pixels_without_depth = sum(np.array(rgb_depth_values) == 0)
if pixels_without_depth > 0:
    print(f"There are {pixels_without_depth} pixels without depth information. Consider increasing `interpolation_size`!")

In [ ]:
valid_landmark_points_3d = np.where(np.array(rgb_depth_values) != 0)[0]

In [ ]:
landmark_points_3d = backproject_points(biwi_loader.get_rgb_intrinsics(), rgb_depth_values, landmarks_img)
landmark_points_3d_render = np.array(landmark_points_3d)
landmark_points_3d_render[:,2] = -landmark_points_3d_render[:,2]  # Invert z-coordinate for easier rendering (landmarks will be right in front of camera)

## 3.1 Restrict Pointcloud to Facial Points 

In [ ]:
face_depth_values = []
face_pixels = []
for x in range(face_pos.left(), face_pos.right() + 1):
    for y in range(face_pos.top(), face_pos.bottom() + 1):
        pixel = [x, y]
        face_depth_value = interpolate_around(rgb_depth_img, pixel, interpolation_size)
        if face_depth_value > 0:
            face_depth_values.append(face_depth_value)
            face_pixels.append(pixel)

In [ ]:
face_pointcloud = backproject_points(biwi_loader.get_rgb_intrinsics(), face_depth_values, face_pixels)
face_pointcloud[:, 2] = -face_pointcloud[:, 2]
face_pointcloud_colors = np.array([img[y, x] for x, y in face_pixels])

In [ ]:
body_depth_values = []
body_pixels = []
for x in range(img_width):
    for y in range(img_height):
        if (x < face_pos.left() or x > face_pos.right()) or (y < face_pos.top() or y > face_pos.bottom()):
            pixel = [x, y]
            body_depth_value = interpolate_around(rgb_depth_img, pixel, interpolation_size)
            if body_depth_value > 0:
                body_depth_values.append(body_depth_value)
                body_pixels.append(pixel)

In [ ]:
body_pointcloud = backproject_points(biwi_loader.get_rgb_intrinsics(), body_depth_values, body_pixels)
body_pointcloud[:, 2] = -body_pointcloud[:, 2]
body_pointcloud_colors = np.array([img[y, x] for x, y in body_pixels])

# 4. ICP

## 4.1 Setup Optimization

In [ ]:
n_params_shape = 20 # 20
n_params_expression = 20 # 10
weight_shape_params = 100 # 10000
weight_expression_params = 100 # 1000

In [ ]:
optimizer = BFMOptimization(bfm, 
                               n_params_shape=n_params_shape,
                               n_params_expression=n_params_expression, 
                               weight_shape_params=weight_shape_params, 
                               weight_expression_params=weight_expression_params)

In [ ]:
initial_camera_pose = np.eye(4) # position camera just in front of face
initial_params = optimizer.create_parameters(
    [0 for _ in range(n_shape_coefficients)],
    [0 for _ in range(n_expression_coefficients)],
    initial_camera_pose
)

## 4.2 Sparse Reconstruction

In [ ]:
l2_regularization_sparse = 10000  # regularizes only face model parameters

In [ ]:
sparse_loss = optimizer.create_sparse_loss_3d(bfm_landmark_indices[valid_landmark_points_3d], landmark_points_3d_render[valid_landmark_points_3d], regularization_strength=l2_regularization_sparse)
result = optimizer.run_optimization(sparse_loss, initial_params)

In [ ]:
params_sparse = optimizer.create_parameters_from_theta(result.x)

## 4.3 Dense Reconstruction with ICP

In [ ]:
nn_mode = NearestNeighborMode.FACE_VERTICES # FACE_VERTICES: every face vertex will be assigned its nearest neighbor in pointcloud
                                            # POINTCLOUD: every point in pointcloud will be assigned its nearest neighbor in face model
distance_type = DistanceType.POINT_TO_POINT
icp_iterations = 2
optimization_steps_per_iteration = 20
l2_regularization_dense = 100 # 100

In [ ]:
params, distances, _ = run_icp(optimizer, 
                               face_pointcloud,
                               bfm, 
                               params_sparse, 
                               max_iterations=icp_iterations, 
                               nearest_neighbor_mode=nn_mode, 
                               distance_type=distance_type,
                               max_nfev=optimization_steps_per_iteration,
                               l2_regularization=l2_regularization_dense)

# 5. Render Face Reconstruction

In [ ]:
params_render = params_sparse

In [ ]:
face_mesh = bfm.draw_sample(
        shape_coefficients=params_render.shape_coefficients, 
        expression_coefficients=params_render.expression_coefficients, 
        color_coefficients=params_render.color_coefficients)

In [ ]:
def setup_scene(show_pointcloud=True, show_mask=True, show_pointcloud_face=False, cut_around_face=4):
    bfm_vertices = params_render.camera_pose @ add_column(face_mesh.vertices, 1).T
    distances, indices = nearest_neighbors(pointcloud, bfm_vertices[:3, :].T)
    pointcloud_mask = distances > cut_around_face
    
    perspective_camera = get_perspective_camera(biwi_loader.get_rgb_intrinsics(), img_width, img_height)
    scene = setup_standard_scene(perspective_camera)
    if show_pointcloud and show_pointcloud_face:
        scene.add(pyrender.Mesh.from_points(pointcloud[pointcloud_mask], colors=colors[pointcloud_mask]), pose=initial_camera_pose)
    if show_mask:
        scene.add(pyrender.Mesh.from_trimesh(bfm.convert_to_trimesh(face_mesh)), pose=params_render.camera_pose)
    if not show_pointcloud and show_pointcloud_face:
        scene.add(pyrender.Mesh.from_points(face_pointcloud, colors=face_pointcloud_colors), pose=initial_camera_pose)
    if show_pointcloud and not show_pointcloud_face:
        scene.add(pyrender.Mesh.from_points(body_pointcloud, colors=body_pointcloud_colors), pose=initial_camera_pose)
    return scene

## 5.1. Interactive 3D Rendering

In [ ]:
scene = setup_scene(show_pointcloud=True, show_mask=True, show_pointcloud_face=True, cut_around_face=8)

In [ ]:
pyrender.Viewer(scene, use_raymond_lighting=True, viewport_size=(img_width, img_height))

## 5.2. Render mask onto Input Image

In [ ]:
scene = setup_scene(show_pointcloud=False, show_mask=True)

In [ ]:
r = pyrender.OffscreenRenderer(img_width, img_height)
color, depth = r.render(scene)
r.delete()

In [ ]:
img_with_mask = np.array(img)
img_with_mask[depth != 0] = color[depth != 0]
plt.imshow(img_with_mask)
plt.show()